-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Streaming Deduplication

In this notebook, you'll learn how to eliminate duplicate records while working with Structured Streaming and Delta Lake. While Spark Structured Streaming provides exactly-once processing guarantees, many source systems will introduce duplicate records, which must be removed in order for joins and updates to produce logically correct results in downstream queries.

## Learning Objectives
By the end of this lesson, you should be able to:
- Apply **`dropDuplicates`** to streaming data
- Use watermarking to manage state information
- Write an insert-only merge to prevent inserting duplicate records into a Delta table
- Use **`foreachBatch`** to perform a streaming upsert

## Setup
Declare a database and set all path variables.

In [0]:
%run ../Includes/Classroom-Setup-4.1

Creating the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_4_1"

Creating date_lookup...(3 seconds / 1,096 records)

Loading batch #1 to the daily stream...(6 seconds, 417,276 records)
Processing the bronze table from the daily stream...(14 seconds / 417,276 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.1
 DA.paths.user_db: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.1/4_1.db
 DA.paths.checkpoints: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.1/_checkpoints
 DA.paths.source_daily: dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.1/streams/daily

Predefined tables in dbacademy_odl_user_771624_databrickslabs_com_adewd_4_1:
 bronze
 date_lookup

Validating datasets...(1 seconds)

Setup completed in 26 seconds

## Identify Duplicate Records

Because Kafka provides at-least-once guarantees on data delivery, all Kafka consumers should be prepared to handle duplicate records.

The de-duplication methods shown here can also be applied when necessary in other parts of your Delta Lake applications.

Let's start by identifying the number of duplicate records in our **`bpm`** topic of the bronze table.

In [0]:
total = (spark.read
              .table("bronze")
              .filter("topic = 'bpm'")
              .count())

print(f"Total: {total:,}")

Total: 417,164

In [0]:
from pyspark.sql import functions as F

json_schema = "device_id LONG, time TIMESTAMP, heartrate DOUBLE"

old_total = (spark.read
                  .table("bronze")
                  .filter("topic = 'bpm'")
                  .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
                  .select("v.*")
                  .dropDuplicates(["device_id", "time"])
                  .count())

print(f"Old Total: {old_total:,}")

Old Total: 347,116

It appears that around 10-20% of our records are duplicates. Note that here we're choosing to apply deduplication at the silver rather than the bronze level. While we are storing some duplicate records, our bronze table retains a history of the true state of our streaming source, presenting all records as they arrived (with some additional metadata recorded). This allows us to recreate any state of our downstream system, if necessary, and prevents potential data loss due to overly aggressive quality enforcement at the initial ingestion as well as minimizing latencies for data ingestion.

## Define a Streaming Read on the Bronze BPM Records

Here we'll bring back in our final logic from our last notebook.

In [0]:
json_schema = "device_id LONG, time TIMESTAMP, heartrate DOUBLE"

bpm_df = (spark.readStream
               .table("bronze")
               .filter("topic = 'bpm'")
               .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
               .select("v.*"))

When dealing with streaming deduplication, there is a level of complexity compared to static data.

As each micro-batch is processed, we need to ensure:
- No duplicate records exist in the microbatch
- Records to be inserted are not already in the target table

Spark Structured Streaming can track state information for the unique keys to ensure that duplicate records do not exist within or between microbatches. Over time, this state information will scale to represent all history. Applying a watermark of appropriate duration allows us to only track state information for a window of time in which we reasonably expect records could be delayed. Here, we'll define that watermark as 30 seconds.

The cell below updates our previous query.

In [0]:
json_schema = "device_id LONG, time TIMESTAMP, heartrate DOUBLE"

deduped_df = (spark.readStream
                   .table("bronze")
                   .filter("topic = 'bpm'")
                   .select(F.from_json(F.col("value").cast("string"), json_schema).alias("v"))
                   .select("v.*")
                   .withWatermark("time", "30 seconds")
                   .dropDuplicates(["device_id", "time"]))

## Insert Only Merge
Delta Lake has optimized functionality for insert-only merges. This operation is ideal for de-duplication: define logic to match on unique keys, and only insert those records for keys that don't already exist.

Note that in this application, we proceed in this fashion because we know two records with the same matching keys represent the same information. If the later arriving records indicated a necessary change to an existing record, we would need to change our logic to include a **`WHEN MATCHED`** clause.

A merge into query is defined in SQL below against a view titled **`stream_updates`**.

In [0]:
sql_query = """
  MERGE INTO heart_rate_silver a
  USING stream_updates b
  ON a.device_id=b.device_id AND a.time=b.time
  WHEN NOT MATCHED THEN INSERT *
"""

## Defining a Microbatch Function for **`foreachBatch`**

The Spark Structured Streaming **`foreachBatch`** method allows users to define custom logic when writing.

The logic applied during **`foreachBatch`** addresses the present microbatch as if it were a batch (rather than streaming) data.

The class defined in the following cell defines simple logic that will allow us to register any SQL **`MERGE INTO`** query for use in a Structured Streaming write.

In [0]:
class Upsert:
    def __init__(self, sql_query, update_temp="stream_updates"):
        self.sql_query = sql_query
        self.update_temp = update_temp 
        
    def upsert_to_delta(self, microBatchDF, batch):
        microBatchDF.createOrReplaceTempView(self.update_temp)
        microBatchDF._jdf.sparkSession().sql(self.sql_query)

Because we're using SQL to write to our Delta table, we'll need to make sure this table exists before we begin.

In [0]:
%sql

CREATE TABLE IF NOT EXISTS heart_rate_silver 
(device_id LONG, time TIMESTAMP, heartrate DOUBLE)
USING DELTA
LOCATION '${da.paths.user_db}/heart_rate_silver'

Now pass the previously defined **`sql_query`** to the **`Upsert`** class.

In [0]:
streaming_merge = Upsert(sql_query)

And then use this class in our **`foreachBatch`** logic.

In [0]:
query = (deduped_df.writeStream
                   .foreachBatch(streaming_merge.upsert_to_delta)
                   .outputMode("update")
                   .option("checkpointLocation", f"{DA.paths.checkpoints}/recordings")
                   .trigger(availableNow=True)
                   .start())

query.awaitTermination()

We can see that our number of unique entries that have been processed to the **`heart_rate_silver`** table matches our batch de-duplication query from above.

In [0]:
new_total = spark.read.table("heart_rate_silver").count()

print(f"Old Total: {old_total:,}")
print(f"New Total: {new_total:,}")

Old Total: 347,116
New Total: 347,116

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
DA.cleanup()

Dropping the database "dbacademy_odl_user_771624_databrickslabs_com_adewd_4_1"
Removing the working directory "dbfs:/user/odl_user_771624@databrickslabs.com/dbacademy/adewd/4.1"
Validating datasets...(1 seconds)

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>